# Лабораторная работа №1

Лабораторную работу необходимо выполнять в ***Google Colab***. Результат работы (в виде ссылки на notebook) выслать письмом на litvinov.vg@ssau.ru. В теме письма указывать ФИО полностью.

Для выполнения задания необходимо подобрать корпус текстов (художественных) на английском языке. Объем корпуса должен составлять не менее $3 \cdot 10^7$ символов. Далее все действия будут выполняться над выбранными данными. Для отладки алгоритмов лучше использовать не весь корпус, а лишь его часть.

In [9]:
CORPUS_PATH = './corpus/eng-uk_web_2002_300K-sentences.txt'
corpus_data = ''
with open(CORPUS_PATH, 'r', encoding="utf-8") as corpus_file:
    corpus_data = corpus_file.read()

In [8]:
corpus_sep = '\n'
corpus_data_test = corpus_sep.join(corpus_data.split(corpus_sep)[:1000])
corpus_data_test

'Consider the extras not usually included with any \'FREE\' car offer: CDW/LDW @ $12.95 to $13.95 a day.\nFor noise which is persistent but occurs on such an irregular basis that the case officer is unable to attend, the Council operates an emergency out of hours service.\nThey may be unable to meet course requirements without the aid of a computer that has a speech synthesis facility.\nOr maybe you know that you\'ll be adding or updating items to your site on a regular basis, weekly, daily or even hourly.\nIt happened to be one of those occasions when he was inundated with phone calls and letters about the great trouble that was going to be caused.\nBeavers live in small family groups, usually consisting of 3-5 individuals and comprising an adult pair, kits, yearlings and one or more sub-adults.\nWith Amelie (Cert 15) Jeunet combines the best of his two previous films to take us into a grown-up fairy tale where a modern day Paris becomes the setting for the growing up and life of AmÃ©

#### Шаг №1
Заменить все числа, которые представлены цифрами, их текстовым представлением (т.е. прописью). (1 = one, 23 = twenty three, 1042 = one thousand forty two, и т.п.). Методы библиотек не использовать, алгоритм необходимо написать самостоятельно.

In [71]:
import re
from collections import defaultdict

numerals = defaultdict(lambda: None, {
    0: 'zero',
    1: 'one',
    2: 'two',
    3: 'three',
    4: 'four',
    5: 'five',
    6: 'six',
    7: 'seven',
    8: 'eight',
    9: 'nine',
    10: 'ten',
    11: 'eleven',
    12: 'twelve',
    13: 'thirteen',
    14: 'fourteen',
    15: 'fifteen',
    16: 'sixteen',
    17: 'seventeen',
    18: 'eighteen',
    19: 'nineteen',
    20: 'twenty',
    30: 'thirty',
    40: 'fourty',
    50: 'fifty',
    60: 'sixty',
    70: 'seventy',
    80: 'eighty',
    90: 'ninety',
    100: 'hundred',
    1000: 'thousand',
    1000000: 'million',
    1000000000: 'billion'
})

numerals_with_endings = defaultdict(lambda: None, {
    0: 'zeroth',
    1: 'first',
    2: 'second',
    3: 'third',
    4: 'fourth',
    5: 'fifth',
    6: 'sixth',
    7: 'seventh',
    8: 'eighth',
    9: 'nineth',
    10: 'tenth',
    11: 'eleventh',
    12: 'twelvth',
    13: 'thirteenth',
    14: 'fourteenth',
    15: 'fifteenth',
    16: 'sixteenth',
    17: 'seventeenth',
    18: 'eighteenth',
    19: 'nineteenth',
    20: 'twentieth',
    30: 'thirtieth',
    40: 'fourtieth',
    50: 'fiftieth',
    60: 'sixtieth',
    70: 'seventieth',
    80: 'eightieth',
    90: 'ninetieth',
    100: 'hundredth',
    1000: 'thousandth',
    1000000: 'millionth',
    1000000000: 'billionth'
})

def transform_num(z, n, d, with_ending):
    result = zeros(z)
    if d is not None:
        result += num_to_numeral(int(n)) + ' point ' + num_to_numeral(int(d[1:]), with_ending)
    else:
        result += num_to_numeral(int(n), with_ending)
    return ' ' + result + ' '
    
def zeros(z):
    res = ''
    if z is not None and len(z) > 0:
        res = 'zero '*len(z)
    return res

def num_to_numeral(n, with_ending=False):
    res = ''
    if not with_ending and numerals[n] is not None:
        res = numerals[n]
    elif with_ending and numerals_with_endings[n] is not None:
        res = numerals_with_endings[n]
    else:
        billions = n // 1000000000
        if billions > 0:
            res += hundreds(billions) + ' billion '
        n = n % 1000000000
        millions = n // 1000000
        if millions > 0:
            res += hundreds(millions) + ' million '
        n = n % 1000000
        thousands = n // 1000
        if thousands > 0:
            res += hundreds(thousands) + ' thousand '
        n = n % 1000
        res += hundreds(n, with_ending)
    return res.rstrip()
  
def hundreds(n, with_ending=False):
    res = ''
    h = n // 100
    if h > 0:
        res += numerals[h] + ' hundred '
    t = n % 100
    if t > 0:
        t_str = numerals[t]
        if t_str is not None:
            res += t_str
        else:
            rest = t % 10
            res += numerals[t - rest] + ' ' + (numerals_with_endings[rest] if with_ending else numerals[rest])
    return res.rstrip()

In [100]:
replaced = re.sub(r"(0*)(\d+)(\.\d+)?([th|st|rd]{2})?", 
       lambda m: transform_num(m.group(1), m.group(2), m.group(3), m.group(4) is not None), 
       corpus_data).replace('  ', ' ').replace('   ', ' ')

#### Шаг №2
Преобразовать текст к нижнему или верхнему регистру.

In [101]:
# вставьте код здесь
replaced = replaced.lower()

#### Шаг №3
Произвести токенезацию (токенами выступают слова), не учитывая знаки препинания (т.е. необходимо разбить текст на токены (лексемы)):
* С помощью nltk.tokenize.RegexpTokenizer (тут необходимо вспомнить регулярные выражения [posix](https://ru.wikibooks.org/wiki/%D0%A0%D0%B5%D0%B3%D1%83%D0%BB%D1%8F%D1%80%D0%BD%D1%8B%D0%B5_%D0%B2%D1%8B%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D1%8F)).

In [126]:
replaced_without_tokens = replaced.replace('.', ' ').replace(',', ' ').replace(':', ' ') \
            .replace(';', ' ').replace('\n', ' ').replace('?', ' ').replace('!', ' ')

In [127]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\s+', gaps=True)

tokenized_by_regexp = tokenizer.tokenize(replaced_without_tokens)
tokenized_by_regexp

['consider',
 'the',
 'extras',
 'not',
 'usually',
 'included',
 'with',
 'any',
 "'free'",
 'car',
 'offer',
 'cdw/ldw',
 '@',
 '$',
 'twelve',
 'point',
 'ninety',
 'five',
 'to',
 '$',
 'thirteen',
 'point',
 'ninety',
 'five',
 'a',
 'day',
 'for',
 'noise',
 'which',
 'is',
 'persistent',
 'but',
 'occurs',
 'on',
 'such',
 'an',
 'irregular',
 'basis',
 'that',
 'the',
 'case',
 'officer',
 'is',
 'unable',
 'to',
 'attend',
 'the',
 'council',
 'operates',
 'an',
 'emergency',
 'out',
 'of',
 'hours',
 'service',
 'they',
 'may',
 'be',
 'unable',
 'to',
 'meet',
 'course',
 'requirements',
 'without',
 'the',
 'aid',
 'of',
 'a',
 'computer',
 'that',
 'has',
 'a',
 'speech',
 'synthesis',
 'facility',
 'or',
 'maybe',
 'you',
 'know',
 'that',
 "you'll",
 'be',
 'adding',
 'or',
 'updating',
 'items',
 'to',
 'your',
 'site',
 'on',
 'a',
 'regular',
 'basis',
 'weekly',
 'daily',
 'or',
 'even',
 'hourly',
 'it',
 'happened',
 'to',
 'be',
 'one',
 'of',
 'those',
 'occasion

* С помощью nltk.word_tokenize.

In [103]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vladislav\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [104]:
from nltk.tokenize import word_tokenize

tokenized_by_method = word_tokenize(replaced_without_tokens)
tokenized_by_method

['consider',
 'the',
 'extras',
 'not',
 'usually',
 'included',
 'with',
 'any',
 "'free",
 "'",
 'car',
 'offer',
 'cdw/ldw',
 '@',
 '$',
 'twelve',
 'point',
 'ninety',
 'five',
 'to',
 '$',
 'thirteen',
 'point',
 'ninety',
 'five',
 'a',
 'day',
 'for',
 'noise',
 'which',
 'is',
 'persistent',
 'but',
 'occurs',
 'on',
 'such',
 'an',
 'irregular',
 'basis',
 'that',
 'the',
 'case',
 'officer',
 'is',
 'unable',
 'to',
 'attend',
 'the',
 'council',
 'operates',
 'an',
 'emergency',
 'out',
 'of',
 'hours',
 'service',
 'they',
 'may',
 'be',
 'unable',
 'to',
 'meet',
 'course',
 'requirements',
 'without',
 'the',
 'aid',
 'of',
 'a',
 'computer',
 'that',
 'has',
 'a',
 'speech',
 'synthesis',
 'facility',
 'or',
 'maybe',
 'you',
 'know',
 'that',
 'you',
 "'ll",
 'be',
 'adding',
 'or',
 'updating',
 'items',
 'to',
 'your',
 'site',
 'on',
 'a',
 'regular',
 'basis',
 'weekly',
 'daily',
 'or',
 'even',
 'hourly',
 'it',
 'happened',
 'to',
 'be',
 'one',
 'of',
 'those',


#### Шаг №4
Произвести частотный анализ слов. Методы библиотек не использовать, т.е. алгоритм необходимо написать самостоятельно. Cохранить результат в выходной csv файл в порядке убывания частот. Формат выходного файла:

| Word | Count |
| --- | --- |
| cat | 1000 |
| dog | 300 |
| .... | .... |
| butterfly | 34 |

In [136]:
import csv

def freq_analysis(tokens, path):
    freq = defaultdict(lambda: 0)
    for t in tokens:
        freq[t] += 1

    with open(path, 'w', newline='', encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Word', 'Count'])
        for k, v in freq.items():
            writer.writerow([k, v])

In [107]:
import pandas as pd

RESULT_PATH = './result/l1_step4.csv'

freq_analysis(tokenized_by_method, RESULT_PATH)
count_df = pd.read_csv(RESULT_PATH)
count_df

,Word,Count
0,consider,676
1,the,348100
2,extras,36
3,not,19557
4,usually,1207
...,...,...
270275,diarrhoea.,1
270276,"""nyquil",1
270277,"nap.""",1
270278,therory,1


#### Шаг №5
Произвести частотный анализ [N-грам](https://en.wikipedia.org/wiki/N-gram) и сохранить в выходной файл в порядке убывания частот (формат файла аналогичен предыдущему примеру, за исключением столбца Word, который заменяется на N-gram. В этой задаче можно использовать готовые библиотеки (смотрим состав библиотеки [NLTK](https://www.nltk.org/)). Формат выходного файла:

| bigram | | Count |
| --- | --- | --- |
| cat | is | 1000 |
| is | a | 300 |
| .... | .... | .... |
| the | fish | 34 |

* Биграм

In [123]:
def ngram_to_csv(ngrams_dict, path):
    with open(path, 'w', newline='', encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Word', *[' ' for _ in range(len(list(ngrams_dict.keys())[0]) - 1)],'Count'])
        for k, v in ngrams_dict.items():
            writer.writerow([*k, v])

In [124]:
from nltk.util import bigrams
from nltk import FreqDist

BIGRAM_PATH = './result/l1_step5_bigram.csv'

bigrams_dict = FreqDist(bigrams(tokenized_by_method))
ngram_to_csv(bigrams_dict, BIGRAM_PATH)
bigram_df = pd.read_csv(BIGRAM_PATH)
bigram_df

,Word,,Count
0,consider,the,155
1,the,extras,10
2,extras,not,1
3,not,usually,30
4,usually,included,2
...,...,...,...
1881421,non-obvious,improvement,1
1881422,existing,invention,1
1881423,invention,he,1
1881424,currently,attracting,1


* 3-грам

In [125]:
from nltk.util import trigrams

TRIGRAM_PATH = './result/l1_step5_trigram.csv'

trigrams_dict = FreqDist(trigrams(tokenized_by_method))
ngram_to_csv(trigrams_dict, TRIGRAM_PATH)
trigram_df = pd.read_csv(TRIGRAM_PATH)
trigram_df

,Word,,.1,Count
0,consider,the,extras,1
1,the,extras,not,1
2,extras,not,usually,1
3,not,usually,included,1
4,usually,included,with,1
...,...,...,...,...
4303878,members,a,week,1
4303879,week,which,proved,1
4303880,proved,the,success,1
4303881,the,us,web,1


#### Шаг №6 (аналогично шагу 3)
Произвести токенезацию, но уже учитывая знаки препинания:
* С помощью nltk.tokenize.RegexpTokenizer.

In [128]:
tokenized_by_regexp_with_punctuation = tokenizer.tokenize(replaced)
tokenized_by_regexp_with_punctuation

['consider',
 'the',
 'extras',
 'not',
 'usually',
 'included',
 'with',
 'any',
 "'free'",
 'car',
 'offer:',
 'cdw/ldw',
 '@',
 '$',
 'twelve',
 'point',
 'ninety',
 'five',
 'to',
 '$',
 'thirteen',
 'point',
 'ninety',
 'five',
 'a',
 'day.',
 'for',
 'noise',
 'which',
 'is',
 'persistent',
 'but',
 'occurs',
 'on',
 'such',
 'an',
 'irregular',
 'basis',
 'that',
 'the',
 'case',
 'officer',
 'is',
 'unable',
 'to',
 'attend,',
 'the',
 'council',
 'operates',
 'an',
 'emergency',
 'out',
 'of',
 'hours',
 'service.',
 'they',
 'may',
 'be',
 'unable',
 'to',
 'meet',
 'course',
 'requirements',
 'without',
 'the',
 'aid',
 'of',
 'a',
 'computer',
 'that',
 'has',
 'a',
 'speech',
 'synthesis',
 'facility.',
 'or',
 'maybe',
 'you',
 'know',
 'that',
 "you'll",
 'be',
 'adding',
 'or',
 'updating',
 'items',
 'to',
 'your',
 'site',
 'on',
 'a',
 'regular',
 'basis,',
 'weekly,',
 'daily',
 'or',
 'even',
 'hourly.',
 'it',
 'happened',
 'to',
 'be',
 'one',
 'of',
 'those',
 '

* С помощью nltk.word_tokenize.

In [132]:
tokenized_by_method_with_punctuation = word_tokenize(replaced)
tokenized_by_method_with_punctuation

KeyboardInterrupt: 

#### Шаг №7 (аналогично шагу 4)
Произвести частотный анализ слов.

In [137]:
FREQ_PUNCT_PATH = './result/l1_step7.csv'

freq_analysis(tokenized_by_regexp_with_punctuation, FREQ_PUNCT_PATH)
count_df = pd.read_csv(FREQ_PUNCT_PATH)
count_df

,Word,Count
0,consider,676
1,the,348100
2,extras,36
3,not,19557
4,usually,1207
...,...,...
270275,diarrhoea.,1
270276,"""nyquil",1
270277,"nap.""",1
270278,therory,1


#### Шаг №8 (аналогично шагу 5)
Произвести частотный анализ N-грам.

* Биграм

In [138]:
BIGRAM_PATH = './result/l1_step8_bigram.csv'

bigrams_dict = FreqDist(bigrams(tokenized_by_regexp_with_punctuation))
ngram_to_csv(bigrams_dict, BIGRAM_PATH)
bigram_df = pd.read_csv(BIGRAM_PATH)
bigram_df

,Word,,Count
0,consider,the,149
1,the,extras,7
2,extras,not,1
3,not,usually,28
4,usually,included,2
...,...,...,...
2193512,non-obvious,improvement,1
2193513,existing,invention.,1
2193514,invention.,he,1
2193515,currently,attracting,1


* 3-грам

In [139]:
TRIGRAM_PATH = './result/l1_step8_trigram.csv'

trigrams_dict = FreqDist(trigrams(tokenized_by_regexp_with_punctuation))
ngram_to_csv(trigrams_dict, TRIGRAM_PATH)
trigram_df = pd.read_csv(TRIGRAM_PATH)
trigram_df

,Word,,.1,Count
0,consider,the,extras,1
1,the,extras,not,1
2,extras,not,usually,1
3,not,usually,included,1
4,usually,included,with,1
...,...,...,...,...
4398367,members,a,week,1
4398368,week,which,proved,1
4398369,proved,the,success,1
4398370,the,us,web,1


#### Шаг №9 (используя результаты 5-го шага)
Произвести генерацию текса на основе частот биграм. 

In [182]:
import operator

bigrams_dict = dict(sorted(dict(bigrams_dict).items(), key=operator.itemgetter(1),reverse=True))

text_list = []
for k, v in bigrams_dict.items():
    for w in k:
        text_list.append(w)
    break
    
found = True
while found:
    found = False
    for k, v in bigrams_dict.items():
        if k[0] == text_list[-1] and k[1] not in text_list:
            text_list.append(k[1])
            found = True
            break
    
result_text = ' '.join(text_list)
result_text

'of the first time to be a new and is not only one thousand nine hundred ninety eight . this was in their own web site for all our clients with an excellent value added that you can also have been made by using your local authorities are available from his wife who had no longer than two , zero point five - twenty four ) or more information on my life as well known about what we will need help us know how much better understanding between them into account when he has now i am sure they were very good practice at least three ; thirty seven : fourty six months after all, but it would like any other people living room where there may find out some cases which could see if you\'re looking forward thinking about. however many years ago she said that, while others interested parties should take place over fifty % discount prices. however, since then please contact details below shows up until recently completed form part ii listed below. "the government departments within easy access contro